In [1]:
import os
import sys
import csv
from csv import reader
import glob
import random

maxInt = sys.maxsize
while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

equal_path = 'data/code_equal'
not_equal_path = 'data/code_not_equal'

equal_tsv = 'data/train_equal.tsv'
not_equal_tsv = 'data/train_real_not_equal.tsv'
fake_not_equal_tsv = 'data/train_fake_not_equal.tsv'

combined_all_tsv = 'data/train_combined_all.tsv'
combined_real_tsv = 'data/train_combined_real.tsv'
combined_fake_tsv = 'data/train_combined_fake.tsv'

## Equal Examples

In [2]:
train_equal_size = 0
equal_list = os.listdir(equal_path)
with open(equal_tsv, 'w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(['code_1', 'code_2', 'label'])
    
    for path in equal_list:
      dir_path = equal_path + '/' + path + '/'
      file_names = glob.glob(dir_path + "*.c")
      file_1 = file_names[0]
      file_2 = file_names[1]
      with open(file_1, 'r') as code_1:
        code_1 = code_1.read().replace('\n', ' ').replace('"','').replace('\\','')
      with open(file_2, 'r') as code_2:
        code_2 = code_2.read().replace('\n', ' ').replace('"','').replace('\\','')
        
      tsv_output.writerow([code_1, code_2, '1'])
      train_equal_size += 1

      if len(file_names) == 3:
        file_3 = file_names[2]
        
        with open(file_3, 'r') as code_3:
          code_3 = code_3.read().replace('\n', ' ').replace('"','').replace('\\','')
          
        tsv_output.writerow([code_1, code_3, '1'])
        tsv_output.writerow([code_2, code_3, '1'])
        train_equal_size += 2

## Real Not-Equal Examples

In [3]:
train_not_equal_size = 0
not_equal_list = os.listdir(not_equal_path)   
with open(not_equal_tsv, 'w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(['code_1', 'code_2', 'label'])
    
    for path in not_equal_list:
      dir_path = not_equal_path + '/' + path + '/'
      file_names = glob.glob(dir_path + "*.c")
      file_1 = file_names[0]
      file_2 = file_names[1]      
      with open(file_1, 'r') as code_1:
        code_1 = code_1.read().replace('\n', ' ').replace('"','').replace('\\','')
      with open(file_2, 'r') as code_2:
        code_2 = code_2.read().replace('\n', ' ').replace('"','').replace('\\','')
        
      tsv_output.writerow([code_1, code_2, '0'])
      train_not_equal_size += 1

      if len(file_names) == 3:
        file_3 = file_names[2]     
        with open(file_3, 'r') as code_3:
          code_3 = code_3.read().replace('\n', ' ').replace('"','').replace('\\','')
          
        tsv_output.writerow([code_1, code_3, '0'])
        tsv_output.writerow([code_2, code_3, '0'])
        train_not_equal_size += 2

## Fake Not-Equal Examples

In [4]:
train_fake_not_equal_size = 0
equal_list = os.listdir(equal_path)
train_fake_not_equal_threshold = train_equal_size - train_not_equal_size

with open(fake_not_equal_tsv, 'w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(['code_1', 'code_2', 'label'])
    
    for i in range(len(equal_list)):
      path_1 = equal_path + '/' + equal_list[i] + '/'
      file_names = glob.glob(path_1 + "*.c")
      file_1 = file_names[0]
      with open(file_1, 'r') as code_1:
        code_1 = code_1.read().replace('\n', ' ').replace('"','').replace('\\','')

      for j in range(i+1, len(equal_list)):
        if train_fake_not_equal_size < train_fake_not_equal_threshold:
          path_2 = equal_path + '/' + equal_list[j] + '/'
          file_names = glob.glob(path_2 + "*.c")
          file_2 = file_names[0]   
          with open(file_2, 'r') as code_2:
            code_2 = code_2.read().replace('\n', ' ').replace('"','').replace('\\','')

          tsv_output.writerow([code_1, code_2, '0'])
          train_fake_not_equal_size += 1

## Combine all into one

In [5]:
with open(combined_all_tsv, 'w', newline='') as f_output_1, open(combined_fake_tsv, 'w', newline='') as f_output_2, open(combined_real_tsv, 'w', newline='') as f_output_3:
  tsv_output_1 = csv.writer(f_output_1, delimiter='\t')
  tsv_output_2 = csv.writer(f_output_2, delimiter='\t')
  tsv_output_3 = csv.writer(f_output_3, delimiter='\t')
  tsv_output_1.writerow(['code_1', 'code_2', 'label'])
  tsv_output_2.writerow(['code_1', 'code_2', 'label'])
  tsv_output_3.writerow(['code_1', 'code_2', 'label'])
  
  with open(equal_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      count = 0
      for row in tsv_reader:
        tsv_output_1.writerow(row)
        count += 1
        if (count <= train_fake_not_equal_size):
          tsv_output_2.writerow(row)
        else:
          tsv_output_3.writerow(row)
  with open(not_equal_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      for row in tsv_reader:
        tsv_output_1.writerow(row)
        tsv_output_3.writerow(row)
  with open(fake_not_equal_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      for row in tsv_reader:
        tsv_output_1.writerow(row)
        tsv_output_2.writerow(row)

## Shuffle & Divide Combined Dataset into Train/Dev/Test: 7/2/1

In [6]:
def shuffle_and_divide(fpath):
  if (fpath == combined_fake_tsv):
    outpath = 'data/dataset_fake/'
  elif (fpath == combined_real_tsv):
    outpath = 'data/dataset_real/'
  else:
    outpath = 'data/dataset_all/'

  if not os.path.exists(os.path.dirname(outpath)):
    try:
      os.makedirs(os.path.dirname(outpath))
    except OSError as exc:
      if exc.errno != errno.EEXIST:
        raise

  with open(outpath+'train.tsv', 'w', newline='') as f_train, open(outpath+'dev.tsv', 'w', newline='') as f_dev, open(outpath+'test.tsv','w', newline='') as f_test:
      tsv_train = csv.writer(f_train, delimiter='\t')
      tsv_dev = csv.writer(f_dev, delimiter='\t')
      tsv_test = csv.writer(f_test, delimiter='\t')

      tsv_train.writerow(['code_1', 'code_2', 'label'])
      tsv_dev.writerow(['code_1', 'code_2', 'label'])
      tsv_test.writerow(['code_1', 'code_2', 'label'])

      fpath_shuffled = os.path.splitext(fpath)[0] + '_shuffled.tsv'
      with open(fpath, 'r') as f_read:
        data = f_read.readlines()
        header, rows = data[0], data[1:]
        random.shuffle(rows)
      with open(fpath_shuffled ,'w') as f_write:
        f_write.write(''.join([header]+rows))
      
      with open(fpath_shuffled, 'r') as f_combined, open(outpath+'test_labels.txt' ,'w') as f_labels:
        tsv_reader = reader(f_combined, delimiter='\t')
        header = next(tsv_reader)
        if header != None:
          i = 0
          for row in tsv_reader:
            if i <= 0.7 * len(rows):
              tsv_train.writerow(row)
            elif i <= 0.9 * len(rows):
              tsv_dev.writerow(row)
            else:
              tsv_test.writerow(row)
              f_labels.write(row[-1] + ' ')
            i += 1

In [7]:
for file in [combined_all_tsv, combined_real_tsv, combined_fake_tsv]:
  shuffle_and_divide(file)